In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## EDA of the Data with PySpark SQL Library 
## Recommendations and Machine Learning using Pyspark ML

<center><h3>The Data is fairly big 3.50 GB and will need Faster Computation by having it in the memory</center>

In [ ]:
# Installing PySpark
!pip install pyspark

In [ ]:
# Some other Dependencies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os 
import re
import json
from os import listdir
from os.path import isfile, join


In [ ]:
# Reading in csv to do some EDA / Training with them 
articles_df=pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
customers_df=pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')
transactions_df=pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')


In [ ]:
# Checking the Dataset in the Tabular Form
print(articles_df.head())


In [ ]:
print(customers_df.head())


In [ ]:
print(transactions_df.head(10))

### Creating the Spark Session

In [ ]:
# Creating a Spark Session
import pyspark
from pyspark.sql import SparkSession
sc=SparkSession.builder.appName('H-M-Recommender-Systesm').config("spark.sql.files.maxPartitionBytes",10000000).getOrCreate()
spark=SparkSession(sc)

### Loading the Files into the Spark DataFrames

In [ ]:
# Importing the data into the Spark DataFrames to do the operations
# articles=spark.read.format("csv").load('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
# customers=spark.read.format("csv").load('../input/h-and-m-personalized-fashion-recommendations/customers.csv')
# transactions=spark.read.format("csv").load('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
# Read with options as we need the Header names as columns
transactions = spark.read.option('header','true').csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
articles = spark.read.option('header','true').csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
customers = spark.read.option('header','true').csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')
# df.printSchema()
articles.printSchema()
customers.printSchema()
transactions.printSchema()
# Here all the schemas of them will be printed

In [ ]:
# Checking the data
transactions.show(5)
articles.show(5)
customers.show(5)

### Importing Spark Libraries

In [ ]:
import pyspark.sql.functions as sql_func
from pyspark.sql.types import *
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics
from pyspark.ml.evaluation import RegressionEvaluator

## Create them as Spark Tables

Registers this DataFrame as a temporary table using the given name.

The lifetime of this temporary table is tied to the SparkSession that was used to create this DataFrame.

This tables will be useful as we will be able to perform SQL Like Queries on them with ease

In [ ]:
transactions.createOrReplaceTempView('transactions')
articles.createOrReplaceTempView('articles')
customers.createOrReplaceTempView('customers')

### Starting off with a Simple SQL Query of Selecting everything from the Table



In [ ]:
query_1 = spark.sql('''
select * 
from transactions
limit 100
''')

query_1.show(10)

In [ ]:
query_2 = spark.sql('''
select * 
from customers
limit 100
''')

query_2.show(10)

In [ ]:
query_3 = spark.sql('''
select * 
from articles
limit 100
''')

query_3.show(10)

### Aggregating to get some statistics 

In [ ]:
# Some statistics from the Transaction Tables
query_4= spark.sql('''
select
  customer_id
  , collect_list(article_id) as article_id_list
  , avg(price) as avg_price
  , max(price) as max_price
  , min(price) as min_price
from transactions
group by customer_id
limit 100
''')

query_4.show(10)

### Getting the Number of Customers below the Age of 25

In [ ]:
query_5=spark.sql('''
select count(customer_id)
from 
customers 
where (customers.age<25) 
''')
query_5.show(10)

### List of Customers that  are Active Club Members

In [ ]:
query_6=spark.sql('''
select count(customer_id)
from customers
where customers.club_member_status=='ACTIVE'
''')
query_6.show()

## Count of Customers that Follow the Fashion News Regularly

In [ ]:
query_7=spark.sql('''
select fashion_news_frequency,count(customer_id)
from customers
GROUP BY fashion_news_frequency
''')
query_7.show()
q7=query_7.toPandas()

#### Plotting the Bar Plot for this query

In [ ]:
f, (ax1) = plt.subplots(1, 1, figsize=(6, 6), sharex=True)
sns.barplot(x=q7['fashion_news_frequency'], y=q7['count(customer_id)'], palette="rocket", ax=ax1)

### Key Customers - Customers that are Active , Follow Fashion News and club members

In [ ]:
query_8=spark.sql('''
select customer_id 
from customers 
where
customers.Active=1.0 AND
customers.fashion_news_frequency=='Regularly' AND
customers.club_member_status=='ACTIVE'

''')
query_8.show()

### Queries on Articles Table 

In [ ]:
# Printing the Dataframe to get an Idea  of the Columns
articles_df.head()

### Get a List of Product Names

In [ ]:
query_9=spark.sql('''
select DISTINCT prod_name
from articles 

''')
query_9.show(50)

### Get a List of Type of Products

In [ ]:
query_10=spark.sql('''
select DISTINCT product_type_name
from articles
''')
query_10.show(100)

### Get a List of Index Names

In [ ]:
query_11=spark.sql('''
select DISTINCT index_name
from articles
''')
query_11.show(20)
# Here we can see these are the Broad 5 Categories

### Get a List of Index group Names

In [ ]:
query_12=spark.sql('''
select DISTINCT
index_group_name
from articles
''')
query_12.show(20)
# Here we can see these are the Broad 5 Categories

In [ ]:
query_12b=spark.sql('''
select index_group_name,count(index_group_name)
from
articles
GROUP BY index_group_name
''')
query_12b=query_12b.toPandas()
sns.barplot(x="count(index_group_name)", y="index_group_name", data=query_12b, color="c")

In [ ]:

query_12a=spark.sql('''
select garment_group_no,count(garment_group_no)
from
articles
GROUP BY garment_group_no
''')
query_12a=query_12a.toPandas()
sns.barplot(x="count(garment_group_no)", y="garment_group_no", data=query_12a, color="b")

### Find Which Index Items are Maximum Present

In [ ]:
query_13=spark.sql('''
select count(article_id)
from
articles
GROUP BY index_group_name
''')
query_13.show()
# This shows which Index Group Name has what Number of Items Present in it

### Sort Which Index has the Most Number of Items

In [ ]:
query_14=spark.sql('''
select index_name, count (article_id)
from 
articles
GROUP BY index_name
''')
query_14.show(10)

### Get the Number of Each Products Present

In [ ]:
# Get a List of Products that are Most in Number
query_15=spark.sql('''
select count(prod_name),prod_name
from 
articles
GROUP BY prod_name
ORDER BY count(prod_name)
DESC
''')
query_15.show()

### Query to Find out which colour Products are present most in the Inventory

In [ ]:
query_16=spark.sql('''
select count(colour_group_name), colour_group_name
from
articles
GROUP BY colour_group_name
ORDER BY count(colour_group_name)
DESC
''')
query_16.show(20)
# From this we can know which colours are present in what quantity in the Inventory

### Queries on the Transaction Table + Joins to Perform Complex Queries

### Printing the Transaction Table to familiarize  with the Features

In [ ]:
transactions_df.head()

### Getting the Count of Number of Transactions through various Sales Id

In [ ]:
query_17=spark.sql('''
select count(sales_channel_id), sales_channel_id
from transactions
GROUP BY sales_channel_id
ORDER BY count(sales_channel_id)
DESC
''')
query_17.show()
# There are only two sales channels and these are the counts corresponding to the sales via each of em

### Get the List of Customers who have done most Number of Purchases

In [ ]:
query_18=spark.sql('''
select customer_id , count(customer_id)
from transactions 
GROUP BY customer_id
ORDER BY count(customer_id) DESC

''')
query_18.show()
# These are the customers who have purchased the Maximum amount of Items

### Get the List of Customers who have done Maximum Amount worth of Purchases

In [ ]:
query_20=spark.sql('''
select customer_id, sum(price)
from 
transactions
GROUP BY transactions.customer_id
ORDER BY (SUM(price)) 
DESC

''')
query_20.show()
# These are the customers that have done the most Amount worth of purchasing from the Store 

### What are the Items that are Most sold by H&M

In [ ]:
query_21=spark.sql('''
select
*
from
transactions as t
inner join articles as ar on ar.article_id = t.article_id
''')
query_21.show(9)

In [ ]:
merged = spark.sql('''
select
  * 
from transactions as tt
left join customers as cs on cs.customer_id = tt.customer_id
left join articles as ar on ar.article_id = tt.article_id
limit 100
''')

print(merged.count())
print(merged.columns)


df_merged = merged.toPandas()

print(df_merged.shape)
df_merged.head()


## Building an ALS Recommender with Pyspark

Collaborative filtering: Collaborative filtering is a method of making automatic predictions (filtering) about the interests of a user by collecting preferences or taste information from many users. Consider example if a person A likes item 1, 2, 3 and B like 2,3,4 then they have similar interests and A should like item 4 and B should like item 1.

Alternating least square(ALS) matrix factorization: The idea is basically to take a large (or potentially huge) matrix and factor it into some smaller representation of the original matrix through alternating least squares. We end up with two or more lower dimensional matrices whose product equals the original one.ALS comes inbuilt in Apache Spark.

### Importing some dependencies

In [ ]:
# Importing some dependencies 
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains
from pyspark.sql import SQLContext 
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import udf,col,when
from pyspark.sql.functions import to_timestamp,date_format
import numpy as np
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import *
from pyspark.sql.functions import min, max
from pyspark.sql.functions import unix_timestamp, lit
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

### Reading Once again in Spark DataFrames

In [ ]:
#  Reading them in PySpark DataFrames
transactionsr = spark.read.option('header','true').csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
articlesr = spark.read.option('header','true').csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
customersr = spark.read.option('header','true').csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')

In [ ]:
min_date, max_date = transactionsr.select(min("t_dat"), max("t_dat")).first()
min_date, max_date

### Segregatng the DateTime column to extract fields of the date time

In [ ]:
# Preparing the Datetime dataset for the Recommender System
hm =  transactionsr.withColumn('t_dat', transactionsr['t_dat'].cast('string'))
hm = hm.withColumn('date', from_unixtime(unix_timestamp('t_dat', 'yyyy-MM-dd')))
hm = hm.withColumn('year', year(col('date')))
hm = hm.withColumn('month', month(col('date')))
hm = hm.withColumn('day', date_format(col('date'), "d"))

hm = hm[hm['year'] == 2020]
hm = hm[hm['month'] == 9]
hm = hm[hm['day'] == 22]
transactionsr.unpersist()

# Prepare the dataset
hm = hm.groupby('customer_id', 'article_id').count()
hm.show(5)

### Getting the total Number of Articles and users and making the sparsity

In [ ]:
# Count the total number of article count in the dataset
numerator = hm.select("count").count()

# Count the number of distinct customerid and distinct articleid
num_users = hm.select("customer_id").distinct().count()
num_articles = hm.select("article_id").distinct().count()

# Set the denominator equal to the number of customer multiplied by the number of articles
denominator = num_users * num_articles

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("Sparsity: ", "%.2f" % sparsity + "%.")

In [ ]:
userId_count = hm.groupBy("customer_id").count().orderBy('count', ascending=False)
userId_count.show()

In [ ]:
articleId_count = hm.groupBy("article_id").count().orderBy('count', ascending=False)
articleId_count.show()

###  Importing some Libraries

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(hm.columns)-set(['count'])) ]
pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(hm).transform(hm)
transformed.show()

In [ ]:
# Preparing the Dataset for Training and Test
(training,test)=transformed.randomSplit([0.8, 0.2])

### Using the Inbuilt ALS Method and fitting the train data on it

In [ ]:
als=ALS(maxIter=5,regParam=0.09,rank=25,userCol="customer_id_index",itemCol="article_id_index",ratingCol="count",coldStartStrategy="drop",nonnegative=True)
model=als.fit(training)

### Evaluating it based on the Regression Evaluator on test data

In [ ]:
evaluator=RegressionEvaluator(metricName="rmse",labelCol="count",predictionCol="prediction")
predictions=model.transform(test)
rmse=evaluator.evaluate(predictions)

### Recommend Items and Users - Collborative Filtering method

In [ ]:
user_recs=model.recommendForAllItems(10).show(10)

In [ ]:
item_recs=model.recommendForAllUsers(10).show(10)

In [ ]:
%%time
userRecsDf = model.recommendForAllUsers(10).cache()
userRecsDf.count()

In [ ]:
userRecsDf.select("customer_id_index","recommendations.article_id_index").show(10,False)

### Testing a Few Classification Models on the Dataset

In [ ]:
qc=spark.sql('''
select customer_id,fashion_news_frequency,club_member_status, Active
from customers
''')
qc.show()
qhm=qc.toPandas()

# # Imputing and Encoding the Data for generating the Heatmap
# qhm['club_member_status'].fillna(value='INACTIVE', inplace=True)
# qhm['Active'].fillna(value=0, inplace=True)
# # qhm['fashion_news_frequency'].replace("NONE", "nf")
# # qhm['fashion_news_frequency'].replace("None", "nf")
# qhm['fashion_news_frequency'].replace({'None': 'NONE'}, regex=True)
# qhm['fashion_news_frequency'].fillna(value='NONE', inplace=True)
# mymap = {'1.0':1}
# qhm.applymap(lambda s: mymap.get(s) if s in mymap else s)
# mymap = {'ACTIVE':1,'INACTIVE':0,'PRE-CREATE':2,'NONE':0,'Regularly':1,'Monthly':2,'LEFT CLUB':3,'1.0':1,'None':0}
# qhm=qhm.applymap(lambda s: mymap.get(s) if s in mymap else s)
# print(qhm['fashion_news_frequency'].unique())
# print(qhm['club_member_status'].unique())
# print(qhm['Active'].unique())
# Label Encoding for the dataframe
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

# Iterative Label Encoding
# for col in qhm.columns:
#     print(col)
#     le.fit(col.values())
#     le.transform(col.values())
qhm.head()

### Classifying based on the Features if they purchase above the Average Shopping Cart - Machine Learning at Scale

### Getting the Average Ticket Size

In [ ]:
ats=spark.sql('''
from transactions select avg(price)
''')
ats.show()


In [ ]:
ats=ats.toPandas()
avg_spend=ats.iloc[0,0]
avg_spend
# ats=ats.toPandas()
# sum_spend=0
# n=0
# for i in range(len(ats)):
#     sum_spend=sum_spend+ats.iloc[i,0]
#     n=n+1
# avg_spend=sum_spend/

### Get the Average Spending of each Customer and the Sum Total of each Customer's purchases

In [ ]:
avg=spark.sql('''
select
  customer_id,
  avg(price) as avg_price, sum(price) as sum_price
from transactions
group by customer_id
 ''')
avg.show(18)

### Registering qc and Avg as Tables to for query operability

In [ ]:
qc.createOrReplaceTempView('qc')
avg.createOrReplaceTempView('avg')

### Ineer Join on the Tables to get the DataFrame

In [ ]:
# Inner Join
avg_spend_cust=spark.sql('''
select avg.customer_id,avg.sum_price,avg.avg_price,qc.fashion_news_frequency,qc.club_member_status,qc.Active
from avg 
INNER JOIN
qc
ON qc.customer_id=avg.customer_id

''')
avg_spend_cust.show(10)

### Getting the DataFrame ready for classification by checking spend > avg or not

In [ ]:
avg_spend_cust.createOrReplaceTempView('main')
from pyspark.sql.functions import when
df = avg_spend_cust.withColumn("avg_price", when(avg_spend_cust.avg_price >= 0.02782927385699682,1) \
      .otherwise(0))
df.show(10)

In [ ]:
df=df.toPandas()

### Imputing the Data

In [ ]:
# Imputing and Encoding the Data for generating the Heatmap
df['club_member_status'].fillna(value='INACTIVE', inplace=True)
# df['Active'].fillna(value=0, inplace=True)
# qhm['fashion_news_frequency'].replace("NONE", "nf")
# qhm['fashion_news_frequency'].replace("None", "nf")
df['fashion_news_frequency'].replace({'None': 'NONE'}, regex=True)
df['fashion_news_frequency'].fillna(value='NONE', inplace=True)
# mymap = {'ACTIVE':1,'INACTIVE':0,'PRE-CREATE':2,'NONE':0,'Regularly':1,'Monthly':2,'LEFT CLUB':3,'1.0':1,'None':0}
# df=df.applymap(lambda s: mymap.get(s) if s in mymap else s)

### Check Categorical Data For Label Encoding

In [ ]:
print(df['fashion_news_frequency'].unique())
print(df['club_member_status'].unique())
print(df['Active'].unique())
df.head()
X=df.drop(['customer_id','avg_price'], axis = 1)
y=df['avg_price']
# from sklearn.model_selection import train_test_split
# # X=df['sum_price','fashion_news_frequency','club_member_status','Active']
# df['avg_price']
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.33, random_state=42)

In [ ]:
from pyspark.sql.types import *

# Auxiliar functions
def equivalent_type(f):
    if f == 'datetime64[ns]': return TimestampType()
    elif f == 'int64': return LongType()
    elif f == 'int32': return IntegerType()
    elif f == 'float64': return DoubleType()
    elif f == 'float32': return FloatType()
    else: return StringType()

def define_structure(string, format_type):
    try: typo = equivalent_type(format_type)
    except: typo = StringType()
    return StructField(string, typo)

# Given pandas dataframe, it will return a spark's dataframe.
def pandas_to_spark(pandas_df):
    columns = list(pandas_df.columns)
    types = list(pandas_df.dtypes)
    struct_list = []
    for column, typo in zip(columns, types): 
      struct_list.append(define_structure(column, typo))
    p_schema = StructType(struct_list)
    return sqlContext.createDataFrame(pandas_df, p_schema)

In [ ]:
df = pandas_to_spark(df)

In [ ]:
# from pyspark.sql import SQLContext
# sqlContext = SQLContext(sc)
# # sqlContext=SQLContext(sparkContext=spark.sparkContext, sparkSession=spark)
# df=sqlContext.createDataFrame(df)

### Constructing Feature  Vectors for Machine Learning

### Vector Assembler + Encoding + Preprocessing

In [ ]:
my_cols = df.select(['avg_price',
 'sum_price',
 'fashion_news_frequency',
 'club_member_status',
 'Active',
])
# my_final_data = my_cols.na.drop()
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)
fnf_indexer = StringIndexer(inputCol='fashion_news_frequency',outputCol='fnfIndex')
fnf_encoder = OneHotEncoder(inputCol='fnfIndex',outputCol='fnfVec')
cms_indexer = StringIndexer(inputCol='club_member_status',outputCol='cmsIndex')
cms_encoder = OneHotEncoder(inputCol='cmsIndex',outputCol='cmsVec')
Active_indexer = StringIndexer(inputCol='Active',outputCol='ActiveIndex')
Active_encoder = OneHotEncoder(inputCol='ActiveIndex',outputCol='ActiveVec')
assembler = VectorAssembler(inputCols=['sum_price',
 'fnfVec',
 'cmsVec',
 'ActiveVec'],outputCol='features')

### Applying classification Algortihms

In [ ]:
# from pyspark.ml.feature import  StringIndexer, VectorAssembler
# categoricalColumns=['fashion_news_frequency','club_member_status','Active']
# numericCols = ['sum_price']
# assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
# assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
from pyspark.ml.classification import LogisticRegression
log_reg = LogisticRegression(featuresCol='features',labelCol='avg_price')
train_data, test_data = df.randomSplit([0.7,.3])
fit_model = log_reg.fit(train_data)
results = fit_model.transform(test_data)
# from pyspark.ml.classification import LogisticRegression
# lr = LogisticRegression(featuresCol = X_trainn, labelCol = y_train, maxIter=10)
# lrModel = lr.fit(X_train)